In [1]:
import pandas as pd
import tensorflow as tf
import scipy as sc
from scipy.spatial import distance
import cv2
import numpy as np
import threading
import time
import datetime
from multiprocessing import Queue
import os
import json
from base64 import b64encode
from typing import (
    Any,
    List,
    Tuple,
)

import align.detect_face    # type: ignore
import facenet              # type: ignore

import matplotlib.pyplot as pl
%matplotlib inline

In [2]:
#base_path = "/mnt/d/nna/"
base_path = "d:/nna/"
minimum_size_of_face = 60

frames = Queue(7*2)
faces = Queue(7*2)

In [3]:
data_path = base_path + "data/"
facenet_path = base_path + "node-facenet/"
model_path = facenet_path + "models/"

In [4]:
class MtcnnBridge():
    """
    MTCNN Face Alignment
    """
    def __init__(self) -> None:
        self.graph = self.session = None            # type: Any
        self.pnet = self.rnet = self.onet = None    # type: Any

        self.minsize = minimum_size_of_face # minimum size of face
        self.threshold = [0.6, 0.7, 0.7]    # three steps's threshold
        self.factor = 0.709                 # scale factor

    def init(self) -> None:
        """ doc """
        self.graph = tf.Graph()
        self.session = tf.Session(graph=self.graph)

        # pylint: disable=not-context-manager
        with self.graph.as_default():
            with self.session.as_default():
                self.pnet, self.rnet, self.onet = \
                    align.detect_face.create_mtcnn(self.session, None)

    def align(
            self,
            image: np.ndarray,
    ) -> Tuple[List[Any], List[Any]]:
        """ doc """

        bounding_boxes, landmarks = align.detect_face.detect_face(
            image,   # get rid of alpha channel(if any)
            self.minsize,
            self.pnet,
            self.rnet,
            self.onet,
            self.threshold,
            self.factor,
        )

        return bounding_boxes.tolist(), landmarks.tolist()

In [5]:
class FacenetBridge(object):
    """
    Bridge of Facenet
    """
    FACENET_MODEL = None   # type: str

    def __init__(self) -> None:
        self.graph = self.session = None        # type: Any

        self.placeholder_input = None           # type: Any
        self.placeholder_phase_train = None     # type: Any
        self.placeholder_embeddings = None      # type: Any

        self.FACENET_MODEL = FacenetBridge.get_model_path()

    def init(self) -> None:
        """ doc """
        self.graph = tf.Graph()
        self.session = tf.Session(graph=self.graph)

        # pylint: disable=not-context-manager
        with self.graph.as_default():
            with self.session.as_default():
                model_dir = os.path.expanduser(self.FACENET_MODEL)
                meta_file, ckpt_file = facenet.get_model_filenames(model_dir)
                saver = tf.train.import_meta_graph(
                    os.path.join(model_dir, meta_file),
                )
                saver.restore(
                    tf.get_default_session(),
                    os.path.join(model_dir, ckpt_file),
                )
                # facenet.load_model(self.FACENET_MODEL)

        self.placeholder_input = self.graph.get_tensor_by_name('input:0')
        self.placeholder_phase_train = \
            self.graph.get_tensor_by_name('phase_train:0')
        self.placeholder_embeddings = \
            self.graph.get_tensor_by_name('embeddings:0')

    @staticmethod
    def get_model_path() -> str:
        return model_path    
    
    def embeddings(
            self,
            images: np.ndarray
    ) -> np.ndarray:
        """
        Get embeddings
        """

        slices = np.empty((len(images), 160, 160, 3))
        for i in range(len(images)):
            slices[i,:,:,:] = facenet.prewhiten(images[i])
            
        feed_dict = {
            self.placeholder_input:         slices,
            self.placeholder_phase_train:   False,
        }
        # Use the facenet model to calcualte embeddings
        embeddings = self.session.run(
            self.placeholder_embeddings,
            feed_dict=feed_dict,
        )

        #print("images " + str(len(images)))
        #print("emb " + str(len(embeddings)))
        
        # Return the only row
        return embeddings

In [6]:
class MtcnnMain(threading.Thread):
    
    def __init__(self):
        threading.Thread.__init__(self)
        self.initNN()

    def initNN(self):
        self.mtcnn = MtcnnBridge()
        self.mtcnn.init()

    def run(self):
        print("Mtcnn thread: run")
        global frames
        global faces
        
        while True:
            #if(not frames.empty()):
                
                frame = frames.get() 
                
                a = datetime.datetime.now()
                areas = self.mtcnn.align(frame[1])[0]                
                areasCount = len(areas)
                
                if (not areasCount == 0):
                    fullSize = frame[0]

                    height = np.size(fullSize, 0)
                    width = np.size(fullSize, 1)

                    positions = np.empty((areasCount, 4))
                    embeddings = np.empty((areasCount, 160, 160, 3))

                    for i in range(areasCount):
                        pos = areas[i]
                        x = max(int(pos[0] * 2), 0)
                        y = max(int(pos[1] * 2), 0)
                        w = min(int(pos[2] * 2), width)
                        h = min(int(pos[3] * 2), height)

                        img = fullSize[y:h, x:w, 0:3]
                        img = cv2.resize(img, dsize=(160, 160), interpolation=cv2.INTER_CUBIC)                                                            

                        positions[i,:] = [x,y,w,h]

                        
                        embeddings[i,:,:,:] = img
                        

                    if (not faces.full()):
                        faces.put([positions, embeddings])
                        
                b = datetime.datetime.now()
                print("Mtcnn: " + str(b - a))
            #else:
                #print("Mtcnn Idle")
                #time.sleep(0.05)
                    

In [7]:
class FacenetMain(threading.Thread):
    
    def __init__(self):
        threading.Thread.__init__(self)
        self.user = []
        self.distances = []        
        self.loadUsers()
        self.initNN()

    def loadUsers(self):
        dataFile = "./data.json"
        file = open(dataFile, "r")
        self.users = json.loads(file.read())
        
    def initNN(self):
        self.face = FacenetBridge()
        self.face.init()

    def trackSessions(self, emb):
        a = datetime.datetime.now()
        nearDistances = []
        for user in self.users:
            userEmb = user['embedding']        
            userName = user['name']

            dist = distance.euclidean(userEmb, emb)
            if dist < 1.0:
                nearDistances.append((userName, dist))
                
        if(len(nearDistances) > 0):
            newlist = sorted(nearDistances, key=lambda x: x[1]) 

            print(newlist[0][0] + " - " + str(newlist[0][1]))
            
            self.distances.append(newlist[0][1])
        b = datetime.datetime.now()
        print("Tracking sessions: " + str(b - a))
        
    def run(self):
        print("Facenet thread: run")
        
        global faces
        
        while True:
            #if(not faces.empty()):
         
                facesOnScreen = faces.get()
                
                a = datetime.datetime.now()
                
                positions = facesOnScreen[0]
                images = facesOnScreen[1]
                                
                embeddings = self.face.embeddings(images)
                
                for emb in embeddings:                    
                    self.trackSessions(emb)      
                    
                b = datetime.datetime.now()   
                print("Facenet: " + str(b - a))
            #else:
                #print("Facenet Idle")
                #time.sleep(0.00)


In [8]:
class ImageGrabber(threading.Thread):
    def __init__(self, ID):
        threading.Thread.__init__(self)
        self.ID=ID
        self.cam=cv2.VideoCapture(ID)

    def run(self):
        print("Image grabber thread: run")
        
        global frames
        
        while True:
            
            ret, frame=self.cam.read()
            
            
            if (not frames.full()):
                a = datetime.datetime.now()
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                height = np.size(frame, 0)
                width = np.size(frame, 1)
                frame2 = cv2.resize(frame, dsize=(int(width / 2), int(height / 2)), interpolation=cv2.INTER_CUBIC)
                
                frames.put([frame, frame2])
                b = datetime.datetime.now()
                print("Grabber: " + str(b - a))
                
            else:
                print('Grabber Idle')
                #time.sleep(0.01)
                
            
                        

In [ ]:
mtcnnCount = 2
facenetCount = 3

mtcnnInstances = []
facenetInstances = []

grabberInstance = ImageGrabber(0)
#grabber = ImageGrabber("rtsp://guest:sup3rGu3st@@10.10.101.156/ch1_1.h264")

for i in range(mtcnnCount):
    mtcnnInstances.append(MtcnnMain())
    
for i in range(facenetCount):
    facenetInstances.append(FacenetMain())

grabberInstance.start()

for mtcnn in mtcnnInstances:
    mtcnn.start()

for fn in facenetInstances:
    fn.start()
    fn.join()

for mtcnn in mtcnnInstances:
    mtcnn.join()

grabberInstance.join()

'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from d:/nna/node-facenet/models/model-20170512-110547.ckpt-250000
'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from d:/nna/node-facenet/models/model-20170512-110547.ckpt-250000
'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from d:/nna/node-facenet/models/model-20170512-110547.ckpt-250000
Image grabber thread: run
Mtcnn thread: run
Mtcnn thread: run
Facenet thread: run
Grabber: 0:00:00.012006
Grabber: 0:00:00.002998
Grabber: 0:00:00.002001
Grabber: 0:00:00.001994
Grabber: 0:00:00.002002
Mtcnn: 0:00:00.123003
Grabber: 0:00:00.001999
Grabber: 0:00:00.003003
Mtcnn: 0:00:00.173997
Grabber: 0:00:00.002002
Mtcnn: 0:00:00.106002
Grabber: 0:00:00.005998
Grabber: 0:00:00.004000
Mtcnn: 0:00:00.

Grabber IdleMtcnn: 0:00:00.125003

Grabber: 0:00:00.003991
Mtcnn: 0:00:00.111993
Oleksii Makarov - 0.901199689466603
Tracking sessions: 0:00:00.014992Grabber: 0:00:00.001990
Facenet: 0:00:00.254999

Grabber Idle
Grabber Idle
Mtcnn: 0:00:00.143997
Grabber: 0:00:00.003001
Mtcnn: 0:00:00.138001
Grabber: 0:00:00.001999
Grabber Idle
Mtcnn: 0:00:00.104001Grabber Idle

Mtcnn: 0:00:00.099004Grabber: 0:00:00.010990

Oleksii Makarov - 0.872354378470929Grabber Idle

Tracking sessions: 0:00:00.017000
Facenet: 0:00:00.253001
Grabber: 0:00:00.001992
Grabber Idle
Mtcnn: 0:00:00.160000
Grabber: 0:00:00.002989Mtcnn: 0:00:00.122000

Grabber Idle
Grabber: 0:00:00.003002
Grabber Idle
Grabber Idle
Oleksii Makarov - 0.8955222248285182Mtcnn: 0:00:00.163994
Grabber Idle
Tracking sessions: 0:00:00.012993
Facenet: 0:00:00.265996

Mtcnn: 0:00:00.147995
Grabber: 0:00:00.004998
Grabber: 0:00:00.009999
Mtcnn: 0:00:00.111996Grabber Idle

Mtcnn: 0:00:00.118003
Grabber: 0:00:00.004999
Grabber: 0:00:00.008006
Grabber I

Grabber: 0:00:00.003006
Grabber Idle
Grabber IdleOleksii Makarov - 0.7750985089563635

Tracking sessions: 0:00:00.017000
Facenet: 0:00:00.283610
Mtcnn: 0:00:00.121999
Mtcnn: 0:00:00.130002
Grabber: 0:00:00.001997
Grabber: 0:00:00.002003
Grabber Idle
Grabber Idle
Mtcnn: 0:00:00.133006
Mtcnn: 0:00:00.126996
Grabber: 0:00:00.009995
Grabber: 0:00:00.004001
Grabber Idle
Oleksii Makarov - 0.8246537357187764
Tracking sessions: 0:00:00.017003Grabber Idle
Facenet: 0:00:00.267995

Mtcnn: 0:00:00.113999
Grabber Idle
Mtcnn: 0:00:00.160000
Grabber: 0:00:00.003997
Mtcnn: 0:00:00.125002
Grabber: 0:00:00.002000
Grabber: 0:00:00.002998
Mtcnn: 0:00:00.139002
Grabber: 0:00:00.002002
Grabber Idle
Grabber Idle
Mtcnn: 0:00:00.138002
Grabber Idle
Oleksii Makarov - 0.8319800437285362
Tracking sessions: 0:00:00.021000
Facenet: 0:00:00.309002
Grabber: 0:00:00.008006
Mtcnn: 0:00:00.133058
Grabber: 0:00:00.014995
Grabber Idle
Grabber Idle
Grabber Idle
Mtcnn: 0:00:00.163059
Mtcnn: 0:00:00.099997
Grabber: 0:00:00.0

Grabber: 0:00:00.022000
Grabber Idle
Grabber Idle
Mtcnn: 0:00:00.138013
Mtcnn: 0:00:00.127001
Grabber: 0:00:00.003005
Oleksii Makarov - 0.8700911400551751Grabber: 0:00:00.019000
Tracking sessions: 0:00:00.022001
Facenet: 0:00:00.314000

Grabber Idle
Grabber Idle
Grabber Idle
Mtcnn: 0:00:00.154995
Mtcnn: 0:00:00.165998
Grabber: 0:00:00.002000
Grabber: 0:00:00.002995
Grabber Idle
Grabber Idle
Mtcnn: 0:00:00.117015
Mtcnn: 0:00:00.109998
Oleksii Makarov - 0.8155757804384587Grabber: 0:00:00.035999

Tracking sessions: 0:00:00.020000
Facenet: 0:00:00.293996
Grabber: 0:00:00.003989
Grabber Idle
Grabber Idle
Grabber Idle
Mtcnn: 0:00:00.140000Mtcnn: 0:00:00.157996

Grabber: 0:00:00.004998
Grabber: 0:00:00.003006
Grabber Idle
Grabber Idle
Mtcnn: 0:00:00.104988
Grabber: 0:00:00.001983
Mtcnn: 0:00:00.139999
Oleksii Makarov - 0.7936372178908826
Tracking sessions: 0:00:00.021000
Facenet: 0:00:00.285994
Grabber: 0:00:00.016004
Mtcnn: 0:00:00.140998
Grabber Idle
Mtcnn: 0:00:00.131999
Grabber: 0:00:00.0

Grabber Idle
Oleksii Makarov - 0.8318190317969216Grabber Idle

Tracking sessions: 0:00:00.012997
Facenet: 0:00:00.236998
Mtcnn: 0:00:00.109011
Mtcnn: 0:00:00.108998
Grabber: 0:00:00.002004
Grabber: 0:00:00.002002
Mtcnn: 0:00:00.090002
Mtcnn: 0:00:00.086999
Grabber: 0:00:00.030005
Grabber: 0:00:00.001999
Grabber Idle
Mtcnn: 0:00:00.080996
Grabber: 0:00:00.008001
Mtcnn: 0:00:00.080001
Grabber: 0:00:00.001995
Oleksii Makarov - 0.8364803597119567
Tracking sessions: 0:00:00.011994
Facenet: 0:00:00.248000
Grabber Idle
Grabber Idle
Mtcnn: 0:00:00.142000
Grabber: 0:00:00.002000
Mtcnn: 0:00:00.128992
Grabber: 0:00:00.003523
Grabber Idle
Mtcnn: 0:00:00.089518
Grabber: 0:00:00.004002
Mtcnn: 0:00:00.089518
Grabber: 0:00:00.007000
Grabber Idle
Oleksii Makarov - 0.8117407644486243
Tracking sessions: 0:00:00.018998
Facenet: 0:00:00.282523Mtcnn: 0:00:00.101001
Grabber Idle

Grabber: 0:00:00.006006
Mtcnn: 0:00:00.158000Grabber Idle

Mtcnn: 0:00:00.134006
Grabber: 0:00:00.045969
Grabber: 0:00:00.002998


...

Grabber: 0:00:00.002995
Grabber: 0:00:00.002999
Grabber Idle
Grabber Idle
Mtcnn: 0:00:00.143999
Mtcnn: 0:00:00.158998
Grabber: 0:00:00.002995
Oleksii Makarov - 0.8004202871561636Grabber: 0:00:00.009998

Tracking sessions: 0:00:00.030028
Facenet: 0:00:00.364006
Grabber Idle
Mtcnn: 0:00:00.175996
Grabber: 0:00:00.001992Mtcnn: 0:00:00.168944

Grabber: 0:00:00.002996
Grabber Idle
Grabber Idle
Mtcnn: 0:00:00.105002
Mtcnn: 0:00:00.106009
Grabber: 0:00:00.002996
Grabber: 0:00:00.001994
Grabber Idle
Grabber Idle
Mtcnn: 0:00:00.137002
Grabber: 0:00:00.010999
Mtcnn: 0:00:00.135973
Grabber Idle
Oleksii Makarov - 0.8036743104291891
Tracking sessions: 0:00:00.033001
Facenet: 0:00:00.373003
Grabber: 0:00:00.008003
Grabber Idle
Grabber Idle
Mtcnn: 0:00:00.162997
Mtcnn: 0:00:00.138001
Grabber: 0:00:00.003000
Grabber: 0:00:00.013999
Mtcnn: 0:00:00.108001
Grabber Idle
Grabber: 0:00:00.007992
Mtcnn: 0:00:00.151000
Grabber: 0:00:00.002997
Oleksii Makarov - 0.8543616537728462Grabber Idle
Grabber Idle

Trac

Mtcnn: 0:00:00.103002
Grabber: 0:00:00.014001
Grabber: 0:00:00.008004
Grabber Idle
Oleksii Makarov - 0.8199157993837766
Grabber Idle
Tracking sessions: 0:00:00.019004
Facenet: 0:00:00.283003
Grabber Idle
Mtcnn: 0:00:00.121998
Grabber IdleMtcnn: 0:00:00.163008

Grabber: 0:00:00.002994
Grabber: 0:00:00.001999
Grabber Idle
Mtcnn: 0:00:00.098003
Mtcnn: 0:00:00.119001
Grabber: 0:00:00.029999
Grabber: 0:00:00.003004
Grabber Idle
Grabber Idle
Mtcnn: 0:00:00.140000
Grabber IdleMtcnn: 0:00:00.133000
Oleksii Makarov - 0.8544484768224451

Tracking sessions: 0:00:00.041994
Facenet: 0:00:00.317997
Grabber: 0:00:00.001992
Grabber: 0:00:00.004995
Mtcnn: 0:00:00.116004
Mtcnn: 0:00:00.136000
Grabber: 0:00:00.007994
Grabber: 0:00:00.059007
Grabber Idle
Mtcnn: 0:00:00.089004
Grabber: 0:00:00.017999Oleksii Makarov - 0.8425667844188152

Tracking sessions: 0:00:00.017002
Facenet: 0:00:00.289000
Grabber Idle
Mtcnn: 0:00:00.213006
Grabber Idle
Grabber: 0:00:00.003001
Mtcnn: 0:00:00.125001
Grabber: 0:00:00.001

Mtcnn: 0:00:00.177008
Grabber: 0:00:00.006003
Mtcnn: 0:00:00.168991
Grabber: 0:00:00.003991
Grabber Idle
Mtcnn: 0:00:00.121004
Grabber Idle
Grabber: 0:00:00.004000
Mtcnn: 0:00:00.115001
Grabber: 0:00:00.003991
Grabber Idle
Grabber IdleOleksii Makarov - 0.8119795930277254

Tracking sessions: 0:00:00.020986
Facenet: 0:00:00.310996
Mtcnn: 0:00:00.121003
Grabber: 0:00:00.001998
Mtcnn: 0:00:00.160999
Grabber: 0:00:00.002001
Grabber Idle
Grabber Idle
Mtcnn: 0:00:00.134999
Grabber: 0:00:00.005000
Grabber Idle
Mtcnn: 0:00:00.156002
Grabber Idle
Mtcnn: 0:00:00.121996
Grabber: 0:00:00.001998
Grabber: 0:00:00.011986
Grabber Idle
Oleksii Makarov - 0.8442488714617067Grabber Idle
Tracking sessions: 0:00:00.016997
Facenet: 0:00:00.398001

Mtcnn: 0:00:00.153001
Grabber: 0:00:00.003002
Grabber Idle
Mtcnn: 0:00:00.187998
Grabber: 0:00:00.001996
Grabber Idle
Mtcnn: 0:00:00.146007
Mtcnn: 0:00:00.095001Grabber: 0:00:00.002996

Grabber: 0:00:00.001999
Grabber Idle
Grabber IdleOleksii Makarov - 0.80868802886

Grabber: 0:00:00.011002
Grabber: 0:00:00.002999
Grabber Idle
Grabber Idle
Mtcnn: 0:00:00.091998
Grabber: 0:00:00.003002
Mtcnn: 0:00:00.116000
Grabber: 0:00:00.009000
Oleksii Makarov - 0.8234817414868723
Tracking sessions: 0:00:00.016001Grabber Idle

Facenet: 0:00:00.276998
Grabber Idle
Mtcnn: 0:00:00.139003
Grabber: 0:00:00.001997
Mtcnn: 0:00:00.145002
Grabber: 0:00:00.002000
Grabber Idle
Mtcnn: 0:00:00.128019Grabber Idle

Mtcnn: 0:00:00.156992
Grabber: 0:00:00.027999
Grabber: 0:00:00.001995
Grabber Idle
Mtcnn: 0:00:00.106003
Grabber: 0:00:00.004002
Oleksii Makarov - 0.8295715978772789Grabber Idle

Tracking sessions: 0:00:00.020004
Facenet: 0:00:00.371003
Grabber Idle
Mtcnn: 0:00:00.150999
Grabber: 0:00:00.006007
Grabber Idle
Grabber Idle
Mtcnn: 0:00:00.202999
Grabber: 0:00:00.002000
Mtcnn: 0:00:00.135998
Grabber: 0:00:00.003003
Grabber Idle
Grabber Idle
Mtcnn: 0:00:00.108005
Oleksii Makarov - 0.8398878118632794Grabber: 0:00:00.001997

Tracking sessions: 0:00:00.024004
Facenet: 0:00:00

Grabber Idle
Mtcnn: 0:00:00.107999
Mtcnn: 0:00:00.084000
Grabber: 0:00:00.022002
Grabber: 0:00:00.009996
Oleksii Makarov - 0.7736336078732597Grabber Idle

Tracking sessions: 0:00:00.022001
Facenet: 0:00:00.295012
Grabber Idle
Mtcnn: 0:00:00.124007
Grabber: 0:00:00.001999
Mtcnn: 0:00:00.151990
Grabber: 0:00:00.006003
Grabber Idle
Mtcnn: 0:00:00.085992
Grabber: 0:00:00.001979
Mtcnn: 0:00:00.099003
Grabber: 0:00:00.002002
Grabber Idle
Grabber Idle
Oleksii Makarov - 0.8219381222093396
Tracking sessions: 0:00:00.026025
Facenet: 0:00:00.284025
Grabber Idle
Mtcnn: 0:00:00.135017
Grabber: 0:00:00.001977
Grabber Idle
Mtcnn: 0:00:00.184999
Grabber Idle
Grabber: 0:00:00.002999
Mtcnn: 0:00:00.133997
Grabber: 0:00:00.038025
Grabber Idle
Mtcnn: 0:00:00.126996
Grabber Idle
Mtcnn: 0:00:00.092000
Grabber: 0:00:00.009001
Grabber: 0:00:00.015997
Oleksii Makarov - 0.7761147626175127
Tracking sessions: 0:00:00.019998Grabber Idle

Facenet: 0:00:00.341001
Mtcnn: 0:00:00.124035Grabber Idle

Grabber: 0:00:00.0